In [0]:
from pyspark.sql.types import * 
from pyspark.sql.functions import *

In [0]:
dbutils.widgets.dropdown('time_period', 'Weekly',['Weekly', 'Monthly'])

In [0]:
from datetime import date, timedelta, datetime
from pyspark.sql.functions import *
time_period = dbutils.widgets.get('time_period')
print(time_period)
today=date.today()

if time_period == 'Weekly':
  start_date = today - timedelta(days=today.weekday(), weeks=1)- timedelta(days=1)
  end_date=start_date+timedelta(days=6)
else:
    first=today.replace(day=1)
    end_date=first-timedelta(days=1)
    start_date=first-timedelta(days=end_date.day)
print(start_date, end_date)

In [0]:
df = spark.read.format('csv').option('inferSchema',True).option('header',True).load('/Volumes/workspace/storeroom/datafiles/superstore.csv')

In [0]:
df.display()

In [0]:
df.createOrReplaceTempView('temp_table')

In [0]:
%sql select * from temp_table

In [0]:
%sql select count(distinct customer_id) as total_customer from temp_table

In [0]:
%sql select count(distinct customer_id) as total_customer from temp_table where order_date between '2024-06-01' and '2024-06-30'

In [0]:
customer_count=spark.sql('select count(distinct Customer_id) as total_customer from temp_table where Order_Date between "'+str(start_date)+'" and "'+str(end_date)+'"')
customer_count.display()

### Total number of orders we have

In [0]:
%sql select count(distinct order_id) as total_customer from temp_table

In [0]:
total_order=spark.sql('select count(distinct order_id) as total_order from temp_table where Order_Date between "'+str(start_date)+'" and "'+str(end_date)+'"')
total_order.display()

### Total Sale and profit

In [0]:
%sql select sum(profit) as total_profit from temp_table

In [0]:
df.printSchema()

In [0]:
my_modified_schema = """
  ID integer,
  Order_id string,
  Order_Date string, 
  Ship_Date date ,
  Ship_Mode string ,
  Customer_id  string,
  Customer_Name string,
  Segment string,
  Country string,
  City string,
  State string,
  Postal_Code integer,
  Region string,
  Product_ID string,
  Category string,
  Sub_Category string,
  Product_Name string,
  Sales double,
  Quantity string,
  Discount string,
  Profit double,
  user_id double,
  state_id double,
  order_s string
"""
df_modified_total_sales = (
    spark.read.format("csv")
    .schema(my_modified_schema)
    .option("header", "true")
    .load("/Volumes/workspace/storeroom/datafiles/superstore.csv")
)

df_modified_total_sales.printSchema()

In [0]:
df_modified_total_sales.display()

In [0]:
df_modified_total_sales.createOrReplaceTempView("temp_table_total_sales")
total_sales=spark.sql('select sum(Sales) from temp_table_total_sales')
total_sales.display()

In [0]:
total_sales_profit=spark.sql('select sum(sales) as total_sales, sum(Profit) as total_profit from temp_table_total_sales')
total_sales_profit.display()

### Top sales by country


In [0]:
%sql select sum(sales) as top_sales, country from temp_table_total_sales group by 2

### Most profitable region and country

In [0]:
%sql select sum(sales) as total_sale, Region, Country  from temp_table_total_sales group by Region, Country